In [0]:
# Load libraries
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from statsmodels.formula.api import ols

In [0]:
# Load data and rename columns
data = pd.read_excel('drive/My Drive/load_weather_dataset.xlsx')
data.columns=['Date','Temp_min','Temp_mean','Temp_max','Load','London_min','London_mean','London_max',
              'Bristol_min','Bristol_mean','Bristol_max','Leeds_min','Leeds_mean','Leeds_max','Lockdown_index']

In [0]:
# Convert to datetime and add day of the week, weekend flag, lockdown and convert Load to log load (base e)
data['Date'] = pd.to_datetime(data['Date'])
data['Day'] = data['Date'].apply(lambda x: x.weekday())
data['Month'] = data['Date'].apply(lambda x: x.month)
data['Weekend'] = data['Day'].apply(lambda x: 1 if x>=5 else 0)
lockdown_date = dt.datetime(2020,3,20) # This is the date we will assume for lockdown
data['Lockdown'] = data['Date'].apply(lambda x: 1 if x > lockdown_date else 0)
data['Load'] = np.log(data['Load'])
data['t'] = (data['Date']-data['Date'][0]) 
data['t'] = data['t'].apply(lambda x: pd.Timedelta.total_seconds(x) / (24*60*60*365)) # Compute t in year fraction
data['t_2'] = data['t']**2 # Compute t^2

# Added forecast for N to enable forecast for N+1 (here I have put the forecast for 27 May, but we need forecast for 28 May ultimately)
data.loc[data['Date']==dt.datetime(2020,5,27),['Load']]=9.889952197763487
data.loc[data['Date']==dt.datetime(2020,5,27),['Temp_mean']]=(18.5 + 18.5 + 15)/3

In [0]:
# Add Bank holiday flags, as energy demand is lower on bank holiday days
bh_ls = ['2017-01-01', '2017-04-14', '2017-05-01', '2017-05-29', '2017-12-25', '2017-12-26',
         '2018-01-01', '2018-03-30','2018-05-07', '2018-12-28', '2018-12-25', '2018-12-26',
         '2019-01-01', '2019-04-19', '2019-04-22', '2019-05-06', '2019-05-27', '2019-12-25', '2019-12-26',
         '2020-01-01', '2020-04-10', '2020-05-08', '2020-05-25']

bd_dt = [pd.to_datetime(x) for x in bh_ls]
data['Holiday'] = data['Date'].apply(lambda x: 1 if x in bd_dt else 0)

In [44]:
data.tail()

,Date,Temp_min,Temp_mean,Temp_max,Load,London_min,London_mean,London_max,Bristol_min,Bristol_mean,Bristol_max,Leeds_min,Leeds_mean,Leeds_max,Lockdown_index,Day,Month,Weekend,Lockdown,t,t_2,Holiday
1239,2020-05-24,11.2,15.000000,18.7,9.755191,12.0,18.0,24.0,11.0,14.5,18.0,12.0,15.0,18.0,66.67,6,5,1,1,3.394521,11.522770,0
1240,2020-05-25,6.6,14.300000,21.9,9.780206,12.0,18.5,25.0,10.0,16.0,22.0,11.0,16.5,22.0,66.67,0,5,0,1,3.397260,11.541377,1
1241,2020-05-26,10.7,16.000000,21.3,9.913885,14.0,20.0,26.0,11.0,16.5,22.0,9.0,15.0,21.0,66.67,1,5,0,1,3.400000,11.560000,0
1242,2020-05-27,NaN,17.333333,NaN,9.889952,11.0,18.5,26.0,12.0,18.5,25.0,8.0,15.0,22.0,66.67,2,5,0,1,3.402740,11.578638,0
1243,2020-05-28,NaN,NaN,NaN,NaN,10.0,16.0,22.0,11.0,18.0,25.0,12.0,18.0,24.0,NaN,3,5,0,1,3.405479,11.597290,0


In [0]:
# Add smoothed series
data['Sma'] = data['Load'].rolling(window=7).mean() # Weekly moving average
data['Ema_10'] = data['Load'].ewm(alpha=0.1).mean() # Exponential smoothing with alpha = 0.1
data['Ema_50'] = data['Load'].ewm(alpha=0.5).mean() # Exponential smoothing with alpha = 0.5
data['Ema_90'] = data['Load'].ewm(alpha=0.9).mean() # Exponential smoothing with alpha = 0.9
data['Ema_99'] = data['Load'].ewm(alpha=0.99).mean() # Exponential smoothing with alpha = 0.99
# Add lags for regression, e.g. Autoregressive(1)
data['Ema_90_1'] = data['Ema_90'].shift(periods = 1)
data['Ema_99_1'] = data['Ema_99'].shift(periods = 1)

In [46]:
# Add seasonal factors (one factor for each day of the week), pre-lockdown
avg = data[data['Date']<lockdown_date]['Ema_90'].mean()

i_factors = data[data['Date']<dt.datetime(2020,3,18)][['Ema_90','Day']].groupby(['Day']).mean().reset_index()
i_factors.rename(columns={'Ema_90':'Avg_Ema_90'},inplace=True)
i_factors['i_t'] = i_factors['Avg_Ema_90'] / avg

# Map this back to the df
data = data.merge(i_factors[['Day','i_t']], on='Day',how='left')
data.head()

,Date,Temp_min,Temp_mean,Temp_max,Load,London_min,London_mean,London_max,Bristol_min,Bristol_mean,Bristol_max,Leeds_min,Leeds_mean,Leeds_max,Lockdown_index,Day,Month,Weekend,Lockdown,t,t_2,Holiday,Sma,Ema_10,Ema_50,Ema_90,Ema_99,Ema_90_1,Ema_99_1,i_t
0,2017-01-01,4.4,5.2,6.0,10.206550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,1,1,0,0.000000,0.000000,1,NaN,10.206550,10.206550,10.206550,10.206550,NaN,NaN,0.989582
1,2017-01-02,-0.6,2.0,4.6,10.294451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0.002740,0.000008,0,NaN,10.252814,10.265151,10.286460,10.293580,10.206550,10.206550,1.001178
2,2017-01-03,-1.2,2.7,6.6,10.437069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0,0.005479,0.000030,0,NaN,10.320804,10.363389,10.422143,10.435634,10.286460,10.293580,1.004336
3,2017-01-04,2.2,4.9,7.6,10.452967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,0,0,0.008219,0.000068,0,NaN,10.359235,10.411164,10.449887,10.452794,10.422143,10.435634,1.004709
4,2017-01-05,-3.2,1.2,5.6,10.494623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,0,0,0.010959,0.000120,0,NaN,10.392296,10.454240,10.490150,10.494205,10.449887,10.452794,1.004320


In [47]:
# Create line graph
cutoff = dt.datetime(2000,1,1)
plt_df = data[data['Date']>=cutoff]
fig = go.Figure()
fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['Load'],
                    mode='lines',
                    name='Log load'))
fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['Ema_10'],
                    mode='lines',
                    name='Smoothed Exp 0.1'))
fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['Ema_90'],
                    mode='lines',
                    name='Smoothed Exp 0.9'))

fig.update_layout(
    title="Log load history",
    xaxis_title="Time",
    yaxis_title="Log load"
)
fig.show()

In [48]:
# Scatter of temperature mean and load
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Temp_mean'], y=data['Ema_90'],
                    mode='markers',
                    text=data['Date'],
                    marker_color=data['Weekend'],
                    name='Log load'))

fig.update_layout(
    title="Load vs mean temperature (weekends in yellow)",
    xaxis_title="Mean temperature",
    yaxis_title="Smoothed log load (exp 0.9)"
)

fig.show()

In [0]:
# If we observe the plot of load vs. temperature, it is non-linear: how to incorporate this effect?
# Naive approach of splitting into two, by inspection:
# Choose 2 knots: T <= 15.5 (linear), T>15.5 (linear)
# We assume it is linear below 15.5 and linear above 15.5 (with different slopes). Since we will be forecasting in late spring / early summer only
# This assumption is probably fine

data['T_mean_15_lt'] = data['Temp_mean'].apply(lambda x: x if x<=15.5 else 0)
data['T_mean_15_bt'] = data['Temp_mean'].apply(lambda x: x if x>15.5 else 0)

In [50]:
# Try basic regression
# Functional form: log x_t = B_0 + B_1 * t + B_2 * t^2 + B_3 * i_t + B_4 * temp + B_5 * s_t-1 + lockdown + e_t

model_0 = ols('Load ~ t + t_2 + i_t + Temp_mean + Ema_90_1 + Lockdown', data=data).fit()
print(model_0.summary())

                            OLS Regression Results                            
Dep. Variable:                   Load   R-squared:                       0.877
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     1461.
Date:                Wed, 27 May 2020   Prob (F-statistic):               0.00
Time:                        13:48:58   Log-Likelihood:                 1835.5
No. Observations:                1242   AIC:                            -3657.
Df Residuals:                    1235   BIC:                            -3621.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.9162      0.296     -9.860      0.0

In [51]:
# From previous step, the time index is not significant, drop

# Functional form: log x_t = B_0 + B_3 * i_t + B_4 * temp + B_5 * s_t-1 + lockdown + e_t

model_1 = ols('Load ~ i_t + Temp_mean + Ema_90_1 + Lockdown', data=data).fit()
print(model_1.summary())

                            OLS Regression Results                            
Dep. Variable:                   Load   R-squared:                       0.874
Model:                            OLS   Adj. R-squared:                  0.874
Method:                 Least Squares   F-statistic:                     2148.
Date:                Wed, 27 May 2020   Prob (F-statistic):               0.00
Time:                        13:48:58   Log-Likelihood:                 1823.6
No. Observations:                1242   AIC:                            -3637.
Df Residuals:                    1237   BIC:                            -3612.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.0209      0.298    -10.153      0.0

In [52]:
# Enhance model_1 with piecewise temperature link
# We previously used a linear model for temperature, let's try the piecewise linear model

model_2 = ols('Load ~ i_t + T_mean_15_lt + T_mean_15_bt +  Ema_90_1 + Lockdown', data=data).fit()
print(model_2.summary())
data['in_forecast_2'] = model_2.predict(data)

                            OLS Regression Results                            
Dep. Variable:                   Load   R-squared:                       0.879
Model:                            OLS   Adj. R-squared:                  0.878
Method:                 Least Squares   F-statistic:                     1791.
Date:                Wed, 27 May 2020   Prob (F-statistic):               0.00
Time:                        13:48:59   Log-Likelihood:                 1846.7
No. Observations:                1242   AIC:                            -3681.
Df Residuals:                    1236   BIC:                            -3651.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.8957      0.293     -9.891   

In [53]:
# Show in-sample forecasts
cutoff = dt.datetime(2018,1,1)
plt_df = data[data['Date']>=cutoff]
fig = go.Figure()
fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['Load'],
                    mode='lines',
                    name='Log load'))
fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['in_forecast_2'],
                    mode='lines',
                    name='Forecast M2'))
fig.show()

In [54]:
# Scatter of observed vs forecast
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Load'], y=data['in_forecast_2'],
                    mode='markers',
                    text=data['Date'],
                    marker=dict(size=5,
                                opacity=0.5),
                    marker_color=data['Weekend'],
                    name='Log load'))
fig.add_trace(go.Scatter(x=data['Load'], y=data['Load'],
                    mode='markers',
                    marker=dict(size=2,
                                color='Gray'),
                    name='y = x'))

fig.update_layout(
    title="In-sample forecast vs observed",
    xaxis_title="Observed log load",
    yaxis_title="Forecast log load"
)

fig.show()

In [55]:
# Forecast one point
model_2.predict({'i_t':i_factors[i_factors['Day']==0]['i_t'], 'T_mean_15_lt':0, 'T_mean_15_bt':20, 'Ema_90_1':10, 'Lockdown':1})[0]

9.889026292103036

In [56]:
np.std(data['in_forecast_2']-data['Load'])

0.05470461161016176

In [57]:
# Inspect errors
# There seems to be some seasonality here - we're too high in Apr-May
# Could it be that daily seasonality is not constant over the months?
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Date'], y=data['in_forecast_2']-data['Load'],
                    mode='lines',
                    name='Log load'))
fig.show()

In [58]:
# Revisiting seasonality factors: want to consider day & month
# We calculate seasonality factors for each day of the week by month (assuming the factors are the same over the years)

avg = data[data['Date']<dt.datetime(2020,3,20)]['Ema_90'].mean()

i_new = data[data['Date']<dt.datetime(2020,3,20)][['Ema_90','Month','Day']].groupby(['Month','Day']).mean().reset_index()
i_new.rename(columns={'Ema_90':'Avg_Ema_90_monthly'},inplace=True)
i_new['i_new'] = i_new['Avg_Ema_90_monthly'] / avg

# Map this back to the df
data = data.merge(i_new[['Day','Month','i_new']], on=['Day','Month'],how='left')
data.head()

,Date,Temp_min,Temp_mean,Temp_max,Load,London_min,London_mean,London_max,Bristol_min,Bristol_mean,Bristol_max,Leeds_min,Leeds_mean,Leeds_max,Lockdown_index,Day,Month,Weekend,Lockdown,t,t_2,Holiday,Sma,Ema_10,Ema_50,Ema_90,Ema_99,Ema_90_1,Ema_99_1,i_t,T_mean_15_lt,T_mean_15_bt,in_forecast_2,i_new
0,2017-01-01,4.4,5.2,6.0,10.206550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,1,1,0,0.000000,0.000000,1,NaN,10.206550,10.206550,10.206550,10.206550,NaN,NaN,0.989582,5.2,0.0,NaN,1.007547
1,2017-01-02,-0.6,2.0,4.6,10.294451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0.002740,0.000008,0,NaN,10.252814,10.265151,10.286460,10.293580,10.206550,10.206550,1.001178,2.0,0.0,10.328331,1.016263
2,2017-01-03,-1.2,2.7,6.6,10.437069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0,0.005479,0.000030,0,NaN,10.320804,10.363389,10.422143,10.435634,10.286460,10.293580,1.004336,2.7,0.0,10.382398,1.018303
3,2017-01-04,2.2,4.9,7.6,10.452967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,0,0,0.008219,0.000068,0,NaN,10.359235,10.411164,10.449887,10.452794,10.422143,10.435634,1.004709,4.9,0.0,10.417487,1.018169
4,2017-01-05,-3.2,1.2,5.6,10.494623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,0,0,0.010959,0.000120,0,NaN,10.392296,10.454240,10.490150,10.494205,10.449887,10.452794,1.004320,1.2,0.0,10.479307,1.020680


In [59]:
# Model M5
# New seasonality factors, include interaction term for hot weekends, holidays and a general factor for lockdown
# The assumption here is that lockdown depresses load however we don't have enough info to specify a better functional form
# Could recompute the seasonality factors accounting for lockdown and not lockdown but not enough data to be able to generalise

model_5 = ols('Load ~ i_new + T_mean_15_lt + T_mean_15_bt +  Ema_90_1 + T_mean_15_bt * Weekend + Holiday + Lockdown', data=data).fit()
print(model_5.summary())
data['in_forecast_5'] = model_5.predict(data)


                            OLS Regression Results                            
Dep. Variable:                   Load   R-squared:                       0.925
Model:                            OLS   Adj. R-squared:                  0.925
Method:                 Least Squares   F-statistic:                     1903.
Date:                Wed, 27 May 2020   Prob (F-statistic):               0.00
Time:                        13:48:59   Log-Likelihood:                 2146.0
No. Observations:                1242   AIC:                            -4274.
Df Residuals:                    1233   BIC:                            -4228.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                3.2114 

In [60]:
# Show in-sample forecasts
cutoff = dt.datetime(2020,1,1)
plt_df = data[data['Date']>=cutoff]
fig = go.Figure()
fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['Load'],
                    mode='lines',
                    name='Log load'))

fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['in_forecast_2'],
                    mode='lines',
                    name='Forecast_M2'))
fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['in_forecast_5'],
                    mode='lines',
                    name='Forecast_M5'))
fig.show()

# Scatter of observed vs forecast
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Load'], y=data['in_forecast_2'],
                    mode='markers',
                    text=data['Date'],
                    marker=dict(size=5,
                                opacity=0.5),
                    marker_color=data['Weekend'],
                    name='Log load'))
fig.add_trace(go.Scatter(x=data['Load'], y=data['Load'],
                    mode='markers',
                    marker=dict(size=2,
                                color='Gray'),
                    name='y = x'))

fig.update_layout(
    title="In-sample forecast vs observed",
    xaxis_title="Observed log load",
    yaxis_title="Forecast log load"
)

fig.show()

# Inspect errors
fig = go.Figure()

fig.add_trace(go.Scatter(x=data['Date'], y=data['in_forecast_2']-data['Load'],
                    mode='lines',
                    name='Error Model 2'))

fig.add_trace(go.Scatter(x=data['Date'], y=data['in_forecast_5']-data['Load'],
                    mode='lines',
                    name='Error Model 5'))
fig.show()

In [64]:
# For prediction

### Update the variables below ###

day = 3 # [Monday = 1, Sunday = 6]
month = 5 # May =5
temp = (16.0 + 18.0 + 18.0)/3 # TODO: We need to figure out how to go from London, Bristol and Leeds weather forecast to one overall forecast
prev_ema = 9.900249 # Look this up from dataframe
is_wkend = 0 # 0 = weekday, 1 = weekend
is_holiday = 0 # 0 = not, 1 = bank holiday
is_lockdown = 1 # 0 = not, 1 = lockdown (still ...)

### Leave this stuff ###

if temp > 15.5:
  temp_2 = temp
  temp_1 = 0
else:
  temp_2 = 0
  temp_1 = temp

i_new_val = float(i_new[(i_new['Day']==day)&(i_new['Month']==month)]['i_new'])

print('The forecast is: '+ str(model_5.predict(pd.DataFrame({'i_new':i_new_val,'T_mean_15_lt':temp_1,'T_mean_15_bt':temp_2, 
                                                             'Ema_90_1':prev_ema, 'Weekend':is_wkend,'Holiday':is_holiday,
                                                             'Lockdown':is_lockdown},index=[0]))[0]))

# For the error, possibly need to split the regression into train and test set and work out the error on the test set and report that
# Not clear how the test set works as technically we are using some exponentially weighted average in the calculation which sees info from all previous steps
# For now using the overall standard deviation

print('Error is: ' + str(np.std(data['in_forecast_5']-data['Load'])))

The forecast is: 9.925255189131438
Error is: 0.04299111032289189


In [62]:
data.tail()

,Date,Temp_min,Temp_mean,Temp_max,Load,London_min,London_mean,London_max,Bristol_min,Bristol_mean,Bristol_max,Leeds_min,Leeds_mean,Leeds_max,Lockdown_index,Day,Month,Weekend,Lockdown,t,t_2,Holiday,Sma,Ema_10,Ema_50,Ema_90,Ema_99,Ema_90_1,Ema_99_1,i_t,T_mean_15_lt,T_mean_15_bt,in_forecast_2,i_new,in_forecast_5
1239,2020-05-24,11.2,15.000000,18.7,9.755191,12.0,18.0,24.0,11.0,14.5,18.0,12.0,15.0,18.0,66.67,6,5,1,1,3.394521,11.522770,0,9.854345,9.873151,9.773883,9.753361,9.754939,9.736891,9.729988,0.989582,15.0,0.000000,9.694268,0.976408,9.710430
1240,2020-05-25,6.6,14.300000,21.9,9.780206,12.0,18.5,25.0,10.0,16.0,22.0,11.0,16.5,22.0,66.67,0,5,0,1,3.397260,11.541377,1,9.831504,9.863856,9.777044,9.777521,9.779953,9.753361,9.754939,1.001178,14.3,0.000000,9.810409,0.985569,9.711938
1241,2020-05-26,10.7,16.000000,21.3,9.913885,14.0,20.0,26.0,11.0,16.5,22.0,9.0,15.0,21.0,66.67,1,5,0,1,3.400000,11.560000,0,9.830242,9.868859,9.845465,9.900249,9.912546,9.777521,9.779953,1.004336,0.0,16.000000,9.860173,0.992897,9.890411
1242,2020-05-27,NaN,17.333333,NaN,9.889952,11.0,18.5,26.0,12.0,18.5,25.0,8.0,15.0,22.0,66.67,2,5,0,1,3.402740,11.578638,0,9.825865,9.870969,9.867709,9.890982,9.890178,9.900249,9.912546,1.004709,0.0,17.333333,9.904462,0.995223,9.929312
1243,2020-05-28,NaN,NaN,NaN,NaN,10.0,16.0,22.0,11.0,18.0,25.0,12.0,18.0,24.0,NaN,3,5,0,1,3.405479,11.597290,0,NaN,9.870969,9.867709,9.890982,9.890178,9.890982,9.890178,1.004320,0.0,0.000000,10.102960,0.994147,10.053586


In [0]:
# Using the data provided as of lunchtime Wednesday 27 (load up to 26 May and weather forecast up to 28 May)
# Need to forecast load for N+1 i.e. 28 May
# Step 1: use the model to forecast load for 27 May, using temperature forecast for 27 May (average of mean of 3 cities); forecast is 9.889952197763487, error 0.043
# Step 2: put the forecast for 27 May into 'Load' for 27 May and re-run (using weather forecast for 28 May, average of 3 city mean)
# Forecast is 9.925255, error: 0.04299
# Submit: Forecast = 9.93, error = 0.04